# Deep learning - project 1
## Creating and training neural network from scratch

In [85]:
import numpy as np
from typing import List, Callable
from abc import ABC, abstractmethod 

In [87]:
class Layer:
    """
    One layer of a neural network
    # attributes: number of neurons
    # implements: operator() which depends on activation function, derivative()
    """
    def __init__(self, input_size: int, output_size: int, activation: Callable[[np.array], np.array], *, sd):
        self.W = np.random.normal(0, sd, size=[input_size, output_size])  # weights
        self.b = np.random.normal(0, sd, size=[1, output_size])  # biases
    
    def __call__(self, input_vector: np.array):
        return activation(input_vector.dot(self.W) + self.b)

class Loss(ABC):
    """
    Base class for a loss function of a network
    # implements: operator(), derivative()
    """
    @abstractmethod
    def __call__(self, y_predicted: np.array, y_true: np.array):
        pass

In [91]:
class QuadraticLoss(Loss):
    """Loss for simple regression: mean squared error."""
    def __call__(self, y_predicted: np.array, y_true: np.array):
        if len(y_predicted) != len(y_true):
            raise IndexError("length of y_predicted ({}) has to be the same as lenght of y_true ({})".format(len(y_predicted), len(y_true)))
        return np.linalg.norm(y_predicted - y_true) / len(y_true)
    
class BernLoss(Loss):
    """
    Loss for binary classification (negative binomial likelihood),
    also known as cross-entropy between the between empirical and model distribution (binomial).
    """
    def __call__(self, y_predicted: np.array, y_true: np.array):
        return -np.mean(
            np.array(
                [np.log(p) if y == 1 else np.log(1-p) 
                 for p, y in zip(y_predicted, y_true)]
            )
        )
    
def sigmoid(x: np.array) -> np.array:
    return 1 / (1 +  np.exp(-x))


In [ ]:
class NNet:
    """Feedforwad (classical) neural network"""
    def __init__(self, layers: List[Layer], loss: Loss):
        self.layers = layers
        self.loss = loss
        
    def forward(x: np.array):
        y = x
        for layer in layers:
            y = layer(y)
        

In [24]:
np.log(1.4)

0.3364722366212129

In [70]:
class Temp:
    def foo(self, a, b, *, c):
        return a + b - c

In [71]:
t = Temp()

In [73]:
t.foo(1, 2, c=3)

0

In [80]:
k = 4
d = 7
x = np.ones([1, k])
W = np.ones([k, d])

In [81]:
x
W

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [82]:
x.dot(W)

array([[4., 4., 4., 4., 4., 4., 4.]])

In [97]:
x = np.ones([2, 3])

In [98]:
x

array([[1., 1., 1.],
       [1., 1., 1.]])

In [99]:
b = np.ones([1, 3])

In [100]:
x + b

array([[2., 2., 2.],
       [2., 2., 2.]])